**WEED INJECTION**

This block performs weed augmentation by cutting weed instances (generated via a fine-tuned Stable Diffusion model and segmented with SAM) and pasting them into background regions of inpainted images.

*Library Imports and Initial Setup*

In [ ]:
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage.transform import AffineTransform, warp
import skimage

FOLDER_AUG = "inpainting_outputs"
FOLDER_MASKS = "combined_masks"
FOLDER_WEEDS_IMG = "segmented_weeds"
FOLDER_WEEDS_MASK = "segmented_weeds_masks"
OUTPUT_FOLDER_IMG = "final_images"
OUTPUT_FOLDER_MASK = "final_masks"
os.makedirs(OUTPUT_FOLDER_MASK, exist_ok=True)
os.makedirs(OUTPUT_FOLDER_IMG, exist_ok=True)

The following block defines core utility functions for augmenting and inserting weed instances into semantic segmentation datasets.

Main functions:
- augment_weed: applies random flips, rotations, and — optionally — geometric transformations (scaling, rotation, shear, translation) to increase weed diversity.
- random_geometric_transform: performs smooth affine transformations on both the weed image and its mask to simulate natural variation.
- crop_to_content: crops the weed instance and its mask to the tightest bounding box around non-zero pixels, optimizing placement.
- find_background_spots: searches for regions in the target mask that contain only background (class 0), ensuring weeds are pasted only in plausible locations.
- paste_weed: blends the weed image onto the target image using alpha transparency and updates the corresponding segmentation mask with label 2 (weed).

In [ ]:
def random_geometric_transform(weed_img, weed_mask):
    transform = AffineTransform(
        scale=(random.uniform(0.8, 1.2), random.uniform(0.8, 1.2)),
        rotation=random.uniform(-0.2, 0.2),
        shear=random.uniform(-0.1, 0.1),
        translation=(random.randint(-5, 5), random.randint(-5, 5))
    )
    
    weed_img = warp(weed_img, transform.inverse, preserve_range=True).astype(np.uint8)
    weed_mask = warp(weed_mask, transform.inverse, preserve_range=True, order=0).astype(np.uint8)
    
    return weed_img, weed_mask

def crop_to_content(weed_img, weed_mask):
    ys, xs = np.where(weed_mask > 0)
    if len(xs) == 0 or len(ys) == 0:
        return None, None
    x_min, x_max = xs.min(), xs.max()
    y_min, y_max = ys.min(), ys.max()
    return weed_img[y_min:y_max+1, x_min:x_max+1], weed_mask[y_min:y_max+1, x_min:x_max+1]

def paste_weed(image, mask, weed_img, weed_mask, position):
    x, y = position
    h, w = weed_img.shape[:2]

    if y + h > image.shape[0] or x + w > image.shape[1]:
        return image, mask

    roi_img = image[y:y+h, x:x+w]
    roi_mask = mask[y:y+h, x:x+w]

    alpha = weed_img[:, :, 3] / 255.0
    for c in range(3):
        roi_img[:, :, c] = (1 - alpha) * roi_img[:, :, c] + alpha * weed_img[:, :, c]

    roi_mask[weed_mask > 0] = 2
    image[y:y+h, x:x+w] = roi_img
    mask[y:y+h, x:x+w] = roi_mask

    return image, mask

def find_background_spots(mask, weed_mask, num_trials=200):
    h_img, w_img = mask.shape
    h_weed, w_weed = weed_mask.shape

    for _ in range(num_trials):
        x = random.randint(0, w_img - w_weed)
        y = random.randint(0, h_img - h_weed)
        region = mask[y:y+h_weed, x:x+w_weed]
        if np.all(region == 0):  
            return x, y
    return None

def augment_weed(weed_img, weed_mask):
    k = random.choice([0, 1, 2, 3])
    weed_img = np.rot90(weed_img, k)
    weed_mask = np.rot90(weed_mask, k)

    if random.random() > 0.5:
        weed_img = np.fliplr(weed_img)
        weed_mask = np.fliplr(weed_mask)

    if random.random() > 0.5:  
        print("Applying random geometric transformation")
        weed_img, weed_mask = random_geometric_transform(weed_img, weed_mask)
    
    return weed_img, weed_mask

This loop iterates through all images in the augmentation folder and randomly injects weed instances into background regions.

In [ ]:
# load weeds
weeds = []
for wf in os.listdir(FOLDER_WEEDS_IMG):
    if not wf.endswith(".png"):
        continue
    path_img = os.path.join(FOLDER_WEEDS_IMG, wf)
    path_mask = os.path.join(FOLDER_WEEDS_MASK, wf.replace(".png", "_mask.png"))

    weed_img = cv2.imread(path_img, cv2.IMREAD_UNCHANGED)
    weed_img = cv2.cvtColor(weed_img, cv2.COLOR_BGRA2RGBA)
    weed_mask = cv2.imread(path_mask, cv2.IMREAD_GRAYSCALE)

    if weed_img is None or weed_mask is None or weed_img.shape[2] != 4:
        continue

    weeds.append((weed_img, weed_mask))


image_files = [f for f in os.listdir(FOLDER_AUG) if f.endswith(".png")]
random.shuffle(image_files)

for filename in image_files:
    print(f"\nProcessing {filename}")
    image_path = os.path.join(FOLDER_AUG, filename)
    mask_path = os.path.join(FOLDER_MASKS, filename.replace(".png", "_mask.png"))

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

    num_weeds = random.randint(0, 1)
    print(f"Adding {num_weeds} weeds...")

    if num_weeds == 1:
        weed_img, weed_mask = random.choice(weeds)
        weed_img, weed_mask = augment_weed(weed_img, weed_mask)
        cropped_img, cropped_mask = crop_to_content(weed_img, weed_mask)
        
        if cropped_img is None:
            continue

        position = find_background_spots(mask, cropped_mask)
        if position:
            image, mask = paste_weed(image, mask, cropped_img, cropped_mask, position)
        else:
            print("No valid position found")
    
    Image.fromarray(image).save(os.path.join(OUTPUT_FOLDER_IMG, filename))
    cv2.imwrite(os.path.join(OUTPUT_FOLDER_MASK, f"{filename.replace('.png', '_mask.png')}"), mask)